<a href="https://colab.research.google.com/github/julesyg/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 55.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=60eb277d6f28d7781bd3ab685141cd05bbf3402991e3c3464f84b39e26902844
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [7]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...|2015-08-31 00:00:00|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...| 

In [8]:
from pyspark.sql.functions import to_date

In [16]:
vine_df = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                N|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            4|          4|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            2|          3|   N|           

In [22]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes 
# count is equal to or greater than 20 to pick reviews that are more likely to be helpful 
# and to avoid having division by zero errors later on.

total_votes_over_20 = vine_df.filter("total_votes>=20")
total_votes_over_20.show()


+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           23|         23|   N|                Y|
|          1|           29|         29|   N|                Y|
|          1|           22|         24|   N|                Y|
|          5|           24|         24|   N|                Y|
|          1|           22|         24|   N|                Y|
|          5|           22|         24|   N|                N|
|          5|          123|        125|   N|                Y|
|          5|           20|         20|   N|                Y|
|          1|           27|         28|   N|                Y|
|          5|           51|         52|   N|                Y|
|          1|           35|         35|   N|                Y|
|          1|           24|         24|   N|                Y|
|          5|           26|         27|   N|           

In [23]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table
# to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

votes_over_50percent = total_votes_over_20.filter(total_votes_over_20["helpful_votes"] / total_votes_over_20["total_votes"] >= .50)
votes_over_50percent.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           23|         23|   N|                Y|
|          1|           29|         29|   N|                Y|
|          1|           22|         24|   N|                Y|
|          5|           24|         24|   N|                Y|
|          1|           22|         24|   N|                Y|
|          5|           22|         24|   N|                N|
|          5|          123|        125|   N|                Y|
|          5|           20|         20|   N|                Y|
|          1|           27|         28|   N|                Y|
|          5|           51|         52|   N|                Y|
|          1|           35|         35|   N|                Y|
|          1|           24|         24|   N|                Y|
|          5|           26|         27|   N|           

In [24]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or 
# table that retrieves all the rows where a review was written 
# as part of the Vine program (paid), vine == 'Y'.

paid_reviews = votes_over_50percent.filter(votes_over_50percent["vine"] == "Y")
paid_reviews.describe().show()

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+------------------+------------------+----+-----------------+
|  count|                21|                21|                21|  21|               21|
|   mean| 4.380952380952381|              36.0|38.095238095238095|null|             null|
| stddev|0.8047478161629565|22.492220877450052| 22.84273355337483|null|             null|
|    min|                 2|                18|                20|   Y|                N|
|    max|                 5|                95|                96|   Y|                N|
+-------+------------------+------------------+------------------+----+-----------------+



In [25]:
# Repeat Step 3, but this time retrieve all the rows where the review 
# was not part of the Vine program (unpaid), vine == 'N'.

unpaid_reviews = votes_over_50percent.filter(votes_over_50percent["vine"] == "N")
unpaid_reviews.describe().show()

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+------------------+------------------+----+-----------------+
|  count|              7689|              7689|              7689|7689|             7689|
|   mean|3.8940044219014176| 36.68305371309663| 39.38574587072441|null|             null|
| stddev|1.5291400195050926|35.628912881666686|37.463200601267104|null|             null|
|    min|                 1|                10|                20|   N|                N|
|    max|                 5|               835|               855|   N|                Y|
+-------+------------------+------------------+------------------+----+-----------------+



In [29]:
# Determine the total number of reviews, the number of 5-star reviews, 
# and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

five_star_paid = paid_reviews[paid_reviews["star_rating"] == 5].count()
total_paid = paid_reviews.count()
percent_paid_five_star = five_star_paid /total_paid *100

print(f"""
Paid Stats
Total: {total_paid}
Five Stars: {five_star_paid}
Percentage of Five Stars: {percent_paid_five_star:.2f}%""")




Paid Stats
Total: 21
Five Stars: 11
Percentage of Five Stars: 52.38%


In [30]:
five_star_unpaid = unpaid_reviews[unpaid_reviews["star_rating"] == 5].count()
total_unpaid = unpaid_reviews.count()
percent_unpaid_five_star = five_star_unpaid /total_unpaid *100

print(f"""
Unpaid Stats
Total: {total_unpaid}
Five Stars: {five_star_unpaid}
Percentage of Five Stars: {percent_unpaid_five_star:.2f}%""")


Unpaid Stats
Total: 7689
Five Stars: 4444
Percentage of Five Stars: 57.80%
